In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
from sklearn.preprocessing import MinMaxScaler
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error

# === File paths ===
data_path = "C:/Users/User/Downloads/PRINCIPLES OF DATA SCIENCE/Datasets/e-commerce-scrubbed-data-filtered.xlsx"
output_dir = "C:/Users/User/Downloads/PRINCIPLES OF DATA SCIENCE/Datasets/Final-output"
os.makedirs(output_dir, exist_ok=True)

# === Load & preprocess ===
df = pd.read_excel(data_path)
df = df.dropna(subset=["category_name_1", "M-Y", "qty_ordered"])
df["M-Y"] = pd.to_datetime(df["M-Y"], errors="coerce")
df = df.dropna(subset=["M-Y"])

# **Keep Weekly Aggregation**
df["Week"] = df["M-Y"].dt.to_period("W-SUN").dt.to_timestamp()
weekly = df.groupby(["Week", "category_name_1"])["qty_ordered"].sum().reset_index()

# **Expanded Outlier Filtering (Top 15%)**
q85 = weekly["qty_ordered"].quantile(0.85)  # Remove top 15% extreme values
weekly = weekly[weekly["qty_ordered"] < q85]

# **Apply Log Scaling for Stability**
weekly["qty_ordered"] = np.log1p(weekly["qty_ordered"])

# **Min-Max Scaling**
scaler = MinMaxScaler()
weekly["qty_ordered"] = scaler.fit_transform(weekly[["qty_ordered"]])

# **Apply Moving Average for Smoothing**
weekly["qty_ordered"] = weekly["qty_ordered"].rolling(window=3, min_periods=1).mean()

# Select top 3 categories by total quantity
top_categories = (
    weekly.groupby("category_name_1")["qty_ordered"].sum().nlargest(3).index.tolist()
)
print(f"Initially selected top 3 categories: {top_categories}")

# Ensure they have enough historical data
train_weeks = 120  # Increased training period
horizon = 6

metrics = []

for cat in top_categories:
    sub = weekly[weekly["category_name_1"] == cat].sort_values("Week").reset_index(drop=True)
    sub.rename(columns={"Week": "ds", "qty_ordered": "y"}, inplace=True)

    # Forecast on available data instead of skipping
    train_df = sub.iloc[:-horizon].copy()
    test_df = sub.iloc[-horizon:].copy()

    model = Prophet(weekly_seasonality=True, changepoint_prior_scale=0.00005)  # Fine-tuned model
    model.fit(train_df)

    forecast = model.predict(test_df[["ds"]])
    df_pred = forecast[["ds", "yhat"]].merge(test_df, on="ds")

    mae = mean_absolute_error(df_pred["y"], df_pred["yhat"])
    rmse = np.sqrt(mean_squared_error(df_pred["y"], df_pred["yhat"]))
    
    # **Switch to MAPE Instead of SMAPE**
    mape = (np.abs(df_pred["y"] - df_pred["yhat"]) / df_pred["y"]).mean() * 100

    metrics.append({"Category": cat, "MAE": mae, "RMSE": rmse, "MAPE (%)": mape})

    # Plot with Improved Readability
    plt.figure(figsize=(12, 6))
    plt.plot(train_df["ds"], train_df["y"], "-o", label="Train Data", color="blue")
    plt.plot(test_df["ds"], test_df["y"], "-o", label="Actual Demand", color="green")
    plt.plot(df_pred["ds"], df_pred["yhat"], "--x", label="Forecasted Demand", color="red")
    
    # **Enhance Readability**
    plt.axvline(train_df["ds"].max(), color="black", linestyle=":", label="Train-Test Split")
    plt.xlabel("Week")
    plt.ylabel("Weekly Demand (Normalized)")
    plt.title(f"{cat}: Weekly Demand Forecast ({train_weeks}w Train / {horizon}w Test)")
    
    # **Grid Lines for Clarity**
    plt.grid(True, linestyle="--", alpha=0.6)

    # **Add Annotation for Highest Peak**
    peak_week = df_pred.loc[df_pred["yhat"].idxmax()]
    plt.annotate("Highest Forecasted Demand", xy=(peak_week["ds"], peak_week["yhat"]), 
                 xytext=(peak_week["ds"], peak_week["yhat"] * 1.1),
                 arrowprops=dict(facecolor='red', shrink=0.05))

    plt.legend()
    plt.tight_layout()

    fig_path = os.path.join(output_dir, f"{cat}_forecast.png")
    plt.savefig(fig_path)
    plt.close()
    print(f"[{cat}] forecast done → {fig_path}")

# === Save summary ===
if metrics:
    dfm = pd.DataFrame(metrics)
    excel_path = os.path.join(output_dir, "forecast_metrics.xlsx")
    dfm.to_excel(excel_path, index=False)

    # Print Summary
    print("\n=== SUMMARY ===")
    print(dfm.to_string(index=False))
    print(f"\nMetrics saved to: {excel_path}")
else:
    print("No forecasts generated.")

Initially selected top 3 categories: ["Men's Fashion", 'Mobiles & Tablets', 'Superstore']


14:05:21 - cmdstanpy - INFO - Chain [1] start processing
14:05:21 - cmdstanpy - INFO - Chain [1] done processing
14:05:22 - cmdstanpy - INFO - Chain [1] start processing


[Men's Fashion] forecast done → C:/Users/User/Downloads/PRINCIPLES OF DATA SCIENCE/Datasets/Final-output\Men's Fashion_forecast.png


14:05:22 - cmdstanpy - INFO - Chain [1] done processing
14:05:23 - cmdstanpy - INFO - Chain [1] start processing


[Mobiles & Tablets] forecast done → C:/Users/User/Downloads/PRINCIPLES OF DATA SCIENCE/Datasets/Final-output\Mobiles & Tablets_forecast.png


14:05:32 - cmdstanpy - INFO - Chain [1] done processing


[Superstore] forecast done → C:/Users/User/Downloads/PRINCIPLES OF DATA SCIENCE/Datasets/Final-output\Superstore_forecast.png

=== SUMMARY ===
         Category      MAE     RMSE  MAPE (%)
    Men's Fashion 0.157888 0.177255 19.686766
Mobiles & Tablets 0.092438 0.105212 11.601516
       Superstore 0.063899 0.071456  7.469586

Metrics saved to: C:/Users/User/Downloads/PRINCIPLES OF DATA SCIENCE/Datasets/Final-output\forecast_metrics.xlsx


In [2]:
print("Forecast dates:", forecast['ds'].tail(10).to_list())
print("Test dates:", test_df['ds'].to_list())

Forecast dates: [Timestamp('2018-02-26 00:00:00'), Timestamp('2018-03-26 00:00:00'), Timestamp('2018-04-30 00:00:00'), Timestamp('2018-05-28 00:00:00'), Timestamp('2018-06-25 00:00:00'), Timestamp('2018-07-30 00:00:00')]
Test dates: [Timestamp('2018-02-26 00:00:00'), Timestamp('2018-03-26 00:00:00'), Timestamp('2018-04-30 00:00:00'), Timestamp('2018-05-28 00:00:00'), Timestamp('2018-06-25 00:00:00'), Timestamp('2018-07-30 00:00:00')]


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130653 entries, 0 to 130652
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   item_id          130653 non-null  int64         
 1   status           130653 non-null  object        
 2   created_at       130653 non-null  datetime64[ns]
 3   sku              130653 non-null  object        
 4   price            130653 non-null  float64       
 5   qty_ordered      130653 non-null  int64         
 6   grand_total      130653 non-null  float64       
 7   increment_id     130653 non-null  int64         
 8   category_name_1  130653 non-null  object        
 9   discount_amount  130653 non-null  float64       
 10  payment_method   130653 non-null  object        
 11  BI Status        130653 non-null  object        
 12  Year             130653 non-null  int64         
 13  Month            130653 non-null  int64         
 14  Customer Since   130